# SIMPLE SEDML VINGNETTE

This notebook provides examples of using ``SimpleSBML``, a Python system for creating and validating Simulation Experiment Description Markup Language (SED-ML) descriptions of computational experiments for models written in the Systems Biology Markup Language (SBML) community standard.

Throughout, we use the term **display variable** to refer to a subset of variables in the model that the user wants to observe (e.g., floating species). A **parameter scan** variable is a parameter of the model that is being varied to see its effect on display variables.

SimpleSEDML provides APIs at two levels. The high level API is task oriented. Four tasks are considered. 
* Time course simulation for a single model. In this task, the user wants to see the interaction between display variables over time.
* Time course simulation for multiple models. Here, the user wants to compare the time courses of the same display variable in different models.
* Parameter scan for a single model. The user wants to see how display values are affected by changes in scan parameters.
* Parameter scan for multiple models. The user wants to compare the effect on the same display variable in different models as a scan parameter is changed.

The above use cases are encapsulated into 4 different API calls with optional parameters related to the simulation and changing values of the paramters of the model. These API calls produce an object with the following methods:
* ``getSEDML`` produces SED-ML for the experiment.
* ``getPhraSEDML`` produces the PhraSED-ML for the experiment.
* ``execute`` returns a ``DataFrame`` and a plot of the variables specified in the constructor.
* ``makeOMEXFile`` creates an omex file

The low level API is essentially a python wrapper for PhraSED-ML, and is available from all objects produced in the high level API.

Below, we illustrate each one of the high level APIs.

# Preliminaries

In [ ]:
# Handle execution locally and in Colab
try:
    import SimpleSEDML as ss
except:
    !pip install SimpleSEDML
    import SimpleSEDML as ss
import pandas as pd

/Users/jlheller/home/Technical/repos/SimpleSEDML/ssed/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
MODEL_ANT = '''
model myModel
    J1: S1 -> S2; k1*S1
    J2: S2 -> S3; k2*S2
    
    S1 = 10
    S2 = 0
    k1 = 1
    k2 = 1

    S1 is "species1"
    S2 is "species2"
end
'''

In [ ]:
MODEL2_ANT = '''
model myModel
    J1: S1 -> S2; k1*S1
    J2: S2 -> S3; k2*S2
    J3: S3 -> S1; k3*S3
    
    S1 = 10
    S2 = 0
    k1 = 1
    k2 = 1
    k3 = 0.1
end
'''

# Use Case 1: Single module time course

## Default parameters

In [ ]:
?ss.makeSingleModelTimeCourse

In [ ]:
smtc = ss.makeSingleModelTimeCourse(MODEL_ANT, title="My Plot", num_point=100)

In [ ]:
print(smtc.getPhraSEDML())

In [ ]:
print(smtc.getSEDML())

We can also look at the PhraSED-ML that gets generated by changing the object that gets returned.

In [ ]:
print(smtc.getPhraSEDML())

And we can execute the SEDML, getting both a plot and a DataFrame of time course values.

In [ ]:
df = smtc.execute()

In [ ]:
df.head()

In [ ]:
# Eliminate the temporary files
smtc.cleanUp()

## Non-default arguments

In [ ]:
WOLF_URL = "https://www.ebi.ac.uk/biomodels/services/download/get-files/MODEL3352181362/3/BIOMD0000000206_url.xml"

In [ ]:
smtc = ss.makeSingleModelTimeCourse(WOLF_URL, num_point=1000)
_ = smtc.execute()

In [ ]:
ss.getModelInformation(WOLF_URL)

In [ ]:
smtc = ss.makeSingleModelTimeCourse(WOLF_URL, model_parameter_dct=dict(k0=5, k1=55),
            title="Wolf with new parameter values")
df = smtc.execute()

In [ ]:
df.head()

In [ ]:
smtc.cleanUp()

# Use Case 2: Multiple Models Time Course

When comparing the time course simulation of multiple models, we want to compare selected **display variables** from the models, either in plots or in a DataFrame. This is done with the  ``MultipleModelTimeCourse`` object.

In [ ]:
mmtc = ss.makeMultipleModelTimeCourse([MODEL_ANT, MODEL2_ANT], model_parameter_dct=dict(k1=10), num_point=100)

In [ ]:
print(mmtc.getSEDML())

In [ ]:
mmtc.execute()

In [ ]:
mmtc.model_dct

In [ ]:
print(mmtc.getPhraSEDML())

## Creating an OMEX file

We can create an OMEX file for this experiment.

In [ ]:
omex_path, _ = mmtc.makeOMEXFile()
omex_path

In [ ]:
# We can examine the content of the OMEX file.
!unzip -l project.omex

In [ ]:
mmtc.cleanUp()

# Use Case 3: Single Model Parameter Scan

In [ ]:
smps = ss.makeSingleModelParameterScan(WOLF_URL, simulation_type="onestep",
                                       project_id="Wolf", title="Wolf",
        time_interval=10, display_variables=["at", "na"], scan_parameter_dct=dict(k1=[50, 550, 5000]))
print(smps.getPhraSEDML())
_ = smps.execute()

# Use Case 4: Multiple Model Parameter Scan

In [ ]:
scan_parameter_df = pd.DataFrame(dict(k1=[50, 550, 5000]))
mmps = ss.makeMultipleModelParameterScan(scan_parameter_df,
                                         model_refs=[WOLF_URL],
                                         simulation_type="onestep",
                                         project_id="Wolf", 
                                         title="Wolf",
                                         time_interval=10, 
                                         display_variables=['at', 'na'])


In [ ]:
# We create additional models and they will be included in the simulations, plots and reports
_ = mmps.addModel("k0_1", WOLF_URL, parameter_dct=dict(k0=1))
_ = mmps.addModel("k0_25", WOLF_URL, parameter_dct=dict(k0=25))
_ = mmps.addModel("k0_600", WOLF_URL, parameter_dct=dict(k0=600))
_ = mmps.addModel("k0_6000", WOLF_URL, parameter_dct=dict(k0=6000))

In [ ]:
#print(smps.getPhraSEDML())
_ = mmps.execute()

# Low Level API

Below, we illustrate use of the low level API in isolation.

In [ ]:
# Make DataFrame of changes to model parameters
parameter_df = pd.DataFrame(dict(k1=[1, 3], k2=[1, 3]))

In [ ]:
ssed = ss.SimpleSEDML()
ssed.addModel("model1", MODEL_ANT, parameter_dct=dict(k1=3), ref_type="ant_str", is_overwrite=True)
ssed.addSimulation("sim1", "uniform", start=0, end=1, num_step=50)
ssed.addTask("task1", "model1", "sim1")
ssed.addRepeatedTask("repeated_task1", "task1", parameter_df=parameter_df)
ssed.addReport("task1.time", "task1.S1", "task1.S2", "task1.S3")
ssed.addPlot("repeated_task1.time", "repeated_task1.S2", title="my plot")
result_df = ssed.execute()

In [ ]:
print(ssed.getSEDML())

In [ ]:
ssed.cleanUp()

# Cleanup

In [ ]:
import os
EXAMPLE_REMOVE_FILES = ["project.omex", "project.omex"]
example_dir = os.getcwd()
remove_files = [os.path.join(example_dir, f) for f in EXAMPLE_REMOVE_FILES]
for ffile in remove_files:
    if os.path.isfile(ffile):
        os.remove(ffile)